## shuffle
### 计算机如何生成素数

只简单介绍一种方法，**线性同余法**

$$N_{j+1} = (A*N_j + B) mod M $$  

其中A,B,M是设定的常数。$N_0$被叫做随机种子，是计算机主板的计数器在内存中的记数值。

从公式中可以看出$N_{j+1}$ 与 $N_j$ 存在线性。

线性同余法生成的是均匀分布，如果需要生成其他分布则是据此进行采样。


### 那么如何随机打乱一个数组

1. 考虑一种方法：对于一个长度为n的数组，每次随机选取两个index，交换这两个index对应的数字。并将他们标记。循环直到数组中所有元素被处理

In [1]:
import random
def shuffle_1(array):
    N = len(array)
    swaped = [False]*N
    while not all(swaped):
        a = random.randrange(N)
        b = random.randrange(N)
        array[a],array[b] = array[b],array[a]
        swaped[b] = True
        swaped[a] = True
test_case = [1,2,3]
shuffle_1(test_case)
print(test_case)

[1, 3, 2]


这个打乱算法是**有问题**的。
- 不一定能退出循环
- 时间复杂度为$O(n)$ 
- 各种组合的概率不均匀

时间复杂度为什么为$O(n)$ 
这个问题实际上可以抽象为在袋子里有n个球的情况下，每次抽2个球，放回抽样，需要抽多少次才能全部抽过一遍。求这个次数的期望
而这个随机变量的期望值与 $n^2$ 有关

In [2]:
def shuffle_2(array):
    "Knuth's algorithm P"
    N = len(array)
    for i in range(N-1):   # 这里N 或者N-1都没有问题。N只是多做一次计算
        j = random.randrange(i,N)
        array[i],array[j] = array[j],array[i]
shuffle_2(test_case)

显然，这个算法的时间复杂度为$O(n)$

In [3]:
from collections import defaultdict

def test_shuffler(shuffler,case="abc",n=100000):
    counter = defaultdict(int)
    for i in range(n):
        case_array = list(case)
        shuffler(case_array)
        counter["".join(case_array)] += 1
    e = n*1.0/factorial(len(case))
    test_ok = all(0.9 <= e/counter[j] <= 1.1 for j in counter)
    name = shuffler.__name__
    if test_ok:
        print("%s ok" %name)
    else:
        print("%s failed" %name)
    for k,v in sorted(counter.items()):
        print(k,v/n)
def factorial(n):
    if n<= 1: return 1
    else:
        return n*factorial(n-1)

In [4]:
test_shuffler(shuffle_1)

shuffle_1 failed
abc 0.04993
acb 0.14037
bac 0.13798
bca 0.26658
cab 0.26639
cba 0.13875


In [5]:
test_shuffler(shuffle_2)

shuffle_2 ok
abc 0.16595
acb 0.16526
bac 0.16748
bca 0.1649
cab 0.1682
cba 0.16821


In [6]:
def shuffle_3(array):
    N = len(array)
    swaped = [False]*N
    while not all(swaped):
        a = random.randrange(N)
        b = random.randrange(N)
        array[a],array[b] = array[b],array[a]
        swaped[a] = True
def shuffle_4(array):
    N = len(array)
    for i in range(N):
        j = random.randrange(N)
        array[i],array[j] = array[j],array[i]

In [7]:
test_shuffler(shuffle_3)
test_shuffler(shuffle_4)

shuffle_3 ok
abc 0.16181
acb 0.16803
bac 0.16925
bca 0.16721
cab 0.16778
cba 0.16592
shuffle_4 failed
abc 0.14805
acb 0.18433
bac 0.18513
bca 0.18578
cab 0.14766
cba 0.14905


## 纸牌游戏


In [8]:
def hand_rank(hand):
    ranks = card_ranks(hand)
    if straight(ranks) and flush(hand):
        return (8,max(ranks))
    elif kind(4,ranks):
        return (7,kind(4,ranks),kind(1,ranks))
    elif kind(3,ranks) and kind(2,ranks):
        return (6,kind(3,ranks),kind(2,ranks))
    elif flush(hand):
        return (5,ranks)
    elif straight(ranks):
        return (4,max(ranks))
    elif kind(3,ranks):
        return(3,kind(3,ranks),ranks)
    elif two_pair(ranks):
        return(2,two_pair(ranks),ranks)
    elif kind(2,ranks):
        return(1,kind(2,ranks),ranks)
    else:
        return(0,ranks)

def card_ranks(hand):
    ranks = ['--23456789TJQKA'.index(r) for r, s in hand]
    ranks.sort(reverse=True)
    return [5,4,3,2,1] if (ranks==[14,5,4,3,2]) else ranks

def flush(hand):
    suits = [s for r,s in hand]
    return len(set(suits))==1

def straight(ranks):
    return (max(ranks)-min(ranks)==4) and len(set(ranks))==5

def kind(n,ranks):
    for r in ranks:
        if ranks.count(r) == n: return r

def two_pair(ranks):
    pair = kind(2,ranks)
    lowpair = kind(2,list(reversed(ranks)))
    if pair and lowpair!= pair:
        return (pair,lowpair)
    else:
        return None

### homework
1. Seven Card Stud

Write a function best_hand(hand) that takes a seven

card hand as input and returns the best possible 5

card hand. The itertools library has some functions

that may help you solve this problem.

In [9]:
def test_best_hand():
    assert (sorted(best_hand("6C 7C 8C 9C TC 5C JS".split()))==["6C","7C","8C","9C","TC"])
    assert (sorted(best_hand("TD TC TH 7C 7D 8C 8S".split()))
            == ['8C', '8S', 'TC', 'TD', 'TH'])
    assert (sorted(best_hand("JD TC TH 7C 7D 7S 7H".split()))
            == ['7C', '7D', '7H', '7S', 'JD'])
    return 'test_best_hand passes'


In [10]:
import itertools
help(itertools)

Help on built-in module itertools:

NAME
    itertools - Functional tools for creating and using iterators.

DESCRIPTION
    Infinite iterators:
    count(start=0, step=1) --> start, start+step, start+2*step, ...
    cycle(p) --> p0, p1, ... plast, p0, p1, ...
    repeat(elem [,n]) --> elem, elem, elem, ... endlessly or up to n times
    
    Iterators terminating on the shortest input sequence:
    accumulate(p[, func]) --> p0, p0+p1, p0+p1+p2
    chain(p, q, ...) --> p0, p1, ... plast, q0, q1, ... 
    chain.from_iterable([p, q, ...]) --> p0, p1, ... plast, q0, q1, ... 
    compress(data, selectors) --> (d[0] if s[0]), (d[1] if s[1]), ...
    dropwhile(pred, seq) --> seq[n], seq[n+1], starting when pred fails
    groupby(iterable[, keyfunc]) --> sub-iterators grouped by value of keyfunc(v)
    filterfalse(pred, seq) --> elements of seq where pred(elem) is False
    islice(seq, [start,] stop [, step]) --> elements from
           seq[start:stop:step]
    starmap(fun, seq) --> fun(*seq

In [11]:
a = itertools.combinations("6C 7C 8C 9C TC 5C JS".split(),5)
list(a)

[('6C', '7C', '8C', '9C', 'TC'),
 ('6C', '7C', '8C', '9C', '5C'),
 ('6C', '7C', '8C', '9C', 'JS'),
 ('6C', '7C', '8C', 'TC', '5C'),
 ('6C', '7C', '8C', 'TC', 'JS'),
 ('6C', '7C', '8C', '5C', 'JS'),
 ('6C', '7C', '9C', 'TC', '5C'),
 ('6C', '7C', '9C', 'TC', 'JS'),
 ('6C', '7C', '9C', '5C', 'JS'),
 ('6C', '7C', 'TC', '5C', 'JS'),
 ('6C', '8C', '9C', 'TC', '5C'),
 ('6C', '8C', '9C', 'TC', 'JS'),
 ('6C', '8C', '9C', '5C', 'JS'),
 ('6C', '8C', 'TC', '5C', 'JS'),
 ('6C', '9C', 'TC', '5C', 'JS'),
 ('7C', '8C', '9C', 'TC', '5C'),
 ('7C', '8C', '9C', 'TC', 'JS'),
 ('7C', '8C', '9C', '5C', 'JS'),
 ('7C', '8C', 'TC', '5C', 'JS'),
 ('7C', '9C', 'TC', '5C', 'JS'),
 ('8C', '9C', 'TC', '5C', 'JS')]

In [12]:
def best_hand(hand):
    combs = itertools.combinations(hand,5)
    comb_rank = [(comb,hand_rank(comb)) for comb in combs]
    comb_rank.sort(reverse=True,key=lambda x : x[1])
    return comb_rank[0][0]

test_best_hand()

'test_best_hand passes'

In [13]:
## answer 
def best_hand(hand):
    return max(itertools.combinations(hand,5), key=hand_rank)

2. wild jokers

Write a function best_wild_hand(hand) that takes as
input a 7-card hand and returns the best 5 card hand.

In this problem, it is possible for a hand to include
jokers. Jokers will be treated as 'wild cards' which
can take any rank or suit of the same color. 

The 
black joker, '?B', can be used as any spade or club
and the red joker, '?R', can be used as any heart 
or diamond.

In [17]:
def best_wild_hand(hand):
    car_res = [hand.copy()]
    if "?R" in hand:
        car_res[0].remove("?R")
        variation = [i+j for i in "23456789TJQKA" for j in "HD"]
        car_res = [j+[i] for i in variation for j in car_res]        
    if "?B" in hand:
        for i in car_res:
            if "?B" in i:
                i.remove("?B")
        variation = [i+j for i in "23456789TJQKA" for j in "SC"]
        car_res = [j+[i] for i in variation for j in car_res]
    results = [max(itertools.combinations(x,5),key= hand_rank) for x in car_res]
    return max(results,key= hand_rank)

In [18]:
def test_best_wild_hand():
    assert (sorted(best_wild_hand("6C 7C 8C 9C TC 5C ?B".split()))
            == ['7C', '8C', '9C', 'JC', 'TC'])
    assert (sorted(best_wild_hand("TD TC 5H 5C 7C ?R ?B".split()))
            == ['7C', 'TC', 'TD', 'TH', 'TS'])
    assert (sorted(best_wild_hand("JD TC TH 7C 7D 7S 7H".split()))
            == ['7C', '7D', '7H', '7S', 'JD'])
    return 'test_best_wild_hand passes'

In [19]:
test_best_wild_hand()

'test_best_wild_hand passes'

## answer 


In [20]:
def best_wild_card(hand):
    hands = set(best_hand(h) for h in itertools.product(*map(replacements,hand)))
    
    return max(hands,key=hand_rank)

def replacements(card):
    if card == "?B": return blackcards
    elif card == "?R": return readcards
    else: return [card]
    

SyntaxError: unexpected EOF while parsing (<ipython-input-20-21566679e6c0>, line 6)